<a href="https://colab.research.google.com/github/BullDF/A4_JSC270/blob/Ethan/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/BullDF/A4_JSC270/blob/main/notebook.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# JSC270 Assignment 4 by Johnny and Ethan

In [47]:
## Import necessary libraries
from typing import List
import pandas as pd
import numpy as np
import spacy
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer
from collections import Counter
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, roc_curve
import matplotlib.pyplot as plt
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Part I: Sentiment Analysis

**a) Consider the training data. What is the balance between the three classes? In other words, what proportion of the observations (in the training set) belong to each class?**

In [48]:
train_url = 'https://raw.githubusercontent.com/3th4novo/JSC270_a4_dataset/main/Corona_NLP_train.csv'
test_url = 'https://raw.githubusercontent.com/3th4novo/JSC270_a4_dataset/main/Corona_NLP_test.csv'

df_train = pd.read_csv(train_url, encoding='latin-1')
df_test = pd.read_csv(test_url, encoding='latin-1')

df_train.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [15]:
df_train['Sentiment'].replace({'Extremely Negative': 'Negative', 'Extremely Positive': 'Positive'}, inplace=True)
df_train['Sentiment'].value_counts(normalize=True)

Positive    0.438467
Negative    0.374128
Neutral     0.187404
Name: Sentiment, dtype: float64

**b) Tokenize the tweets. In other words, for each observation, convert the tweet from a single string of running text into a list of individual tokens (possibly with punctuation), splitting on whitespace. The result should be that each observation (tweet) is a list of individual tokens.**

In [49]:
def tokenize(tweet):
   return tweet.split()

df_train['tokens'] = df_train['OriginalTweet'].apply(tokenize)

print(df_train['tokens'].head(5))

0    [@MeNyrbie, @Phil_Gahan, @Chrisitv, https://t....
1    [advice, Talk, to, your, neighbours, family, t...
2    [Coronavirus, Australia:, Woolworths, to, give...
3    [My, food, stock, is, not, the, only, one, whi...
4    [Me,, ready, to, go, at, supermarket, during, ...
Name: tokens, dtype: object


**c) Using a regular expression, remove any URL tokens from each of the observations.**

In [50]:
def remove_url(lst):
  result = [item for item in lst if not re.search('http\S+|www\.\S+', item)]
  return result

df_train['tokens_no_url'] = df_train['tokens'].apply(remove_url)
print(df_train['tokens_no_url'].head(5))

0        [@MeNyrbie, @Phil_Gahan, @Chrisitv, and, and]
1    [advice, Talk, to, your, neighbours, family, t...
2    [Coronavirus, Australia:, Woolworths, to, give...
3    [My, food, stock, is, not, the, only, one, whi...
4    [Me,, ready, to, go, at, supermarket, during, ...
Name: tokens_no_url, dtype: object


**d) Remove all punctuation (,.?!;:’") and special characters(@, #, +, &, =, $, etc). Also, convert all tokens to lowercase only. Can you think of a scenario when you might want to keep some forms of punctuation?**

In [51]:
def remove_punc_spec_lower(lst):
    result = [re.sub('[^\w\s]', '', item) for item in lst]
    lowered = [item.lower() for item in result]
    return lowered

df_train['tokens_no_punc'] = df_train['tokens_no_url'].apply(remove_punc_spec_lower)
print(df_train['tokens_no_punc'].head(5))

0           [menyrbie, phil_gahan, chrisitv, and, and]
1    [advice, talk, to, your, neighbours, family, t...
2    [coronavirus, australia, woolworths, to, give,...
3    [my, food, stock, is, not, the, only, one, whi...
4    [me, ready, to, go, at, supermarket, during, t...
Name: tokens_no_punc, dtype: object


**e) Now stem your tokens. This will have the effect of converting similar word forms into identical tokens (e.g. run, runs, running → run). Please specify which stemmer you use.**

In [52]:
stemmer = PorterStemmer()

def stemming(lst):
  token_stems = [stemmer.stem(w) for w in lst]
  return token_stems

df_train['tokens_stemmed'] = df_train['tokens_no_punc'].apply(stemming)
print(df_train['tokens_stemmed'].head(5))

0            [menyrbi, phil_gahan, chrisitv, and, and]
1    [advic, talk, to, your, neighbour, famili, to,...
2    [coronaviru, australia, woolworth, to, give, e...
3    [my, food, stock, is, not, the, onli, one, whi...
4    [me, readi, to, go, at, supermarket, dure, the...
Name: tokens_stemmed, dtype: object


**f) Lastly, remove stopwords. Using the english stopwords list from nltk, remove these common words from your observations. This list is very long (I think almost 200 words), so remove only the first 100 stopwords in the list.**

In [53]:
sw = stopwords.words('english')[0:100]

def remove_sw(lst):
  tokens_no_sw = [w for w in lst if w not in sw]
  return tokens_no_sw

df_train['stems_no_sw'] = df_train['tokens_stemmed'].apply(remove_sw)
print(df_train['stems_no_sw'].head(5))

0                      [menyrbi, phil_gahan, chrisitv]
1    [advic, talk, neighbour, famili, exchang, phon...
2    [coronaviru, australia, woolworth, give, elder...
3    [food, stock, not, onli, one, empti, pleas, do...
4    [readi, go, supermarket, dure, covid19, outbre...
Name: stems_no_sw, dtype: object


**g) Now convert your lists of words into vectors of word counts. You may find Scikit-learn’s CountVectorizer useful here. What is the length of your vocabulary?**

In [54]:
# Separate labels from features, converting to numpy arrays
encode = {'Extremely Negative': 0, 'Negative': 0, 'Neutral': 1, 'Positive':2, 'Extremely Positive': 2}
df_train['Sentiment_num'] = df_train['Sentiment'].map(encode)
x_train, y_train = df_train['stems_no_sw'].to_numpy(), df_train['Sentiment_num'].to_numpy()

def override_fcn(doc):
  # We expect a list of tokens as input
  return doc

# Count Vectorizer
count_vec = CountVectorizer(
    analyzer='word',
    tokenizer= override_fcn,
    preprocessor= override_fcn,
    token_pattern= None,
    max_features = 10000)

counts_train = count_vec.fit_transform(x_train) # count matrix for the train data
print(counts_train.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


In [55]:
# I think this should be the size of volcabulary
print(len(counts_train.toarray()[0]))

10000


In [56]:
print(count_vec.vocabulary_)

{'advic': 528, 'talk': 8666, 'neighbour': 6013, 'famili': 3494, 'exchang': 3394, 'phone': 6676, 'number': 6173, 'creat': 2426, 'contact': 2102, 'list': 5300, 'school': 7716, 'employ': 3231, 'chemist': 1761, 'gp': 4088, 'set': 7834, 'onlin': 6292, 'shop': 7907, 'account': 471, 'adequ': 506, 'suppli': 8563, 'regular': 7316, 'med': 5623, 'not': 6143, 'order': 6330, 'coronaviru': 2200, 'australia': 941, 'woolworth': 9744, 'give': 3993, 'elderli': 3191, 'disabl': 2834, 'dedic': 2643, 'hour': 4471, 'amid': 689, 'covid19': 2328, 'outbreak': 6357, 'food': 3716, 'stock': 8411, 'onli': 6291, 'one': 6285, 'empti': 3235, 'pleas': 6757, 'dont': 2967, 'panic': 6480, 'there': 8807, 'will': 9692, 'enough': 3260, 'everyon': 3369, 'take': 8657, 'more': 5830, 'than': 8782, 'need': 6003, 'stay': 8337, 'calm': 1538, 'safe': 7593, 'covid19fr': 2335, 'covid_19': 2374, 'confin': 2047, 'readi': 7225, 'go': 4027, 'supermarket': 8548, 'dure': 3075, 'becaus': 1125, 'im': 4590, 'paranoid': 6505, 'seriou': 7826, 't

**h) Fit a Naive Bayes model to your data. Report the training and test error of the model. Use accuracy as the error metric. Also, report the 5 most probable words in each class, along with their counts. You might find Scikit-learn’s MultinomialNB() transformer useful. Use Laplace smoothing to prevent probabilities of zero.**

In [23]:
df_test.head(5)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [ ]:
print(df_test['Sentiment'].unique())

['Extremely Negative' 'Positive' 'Extremely Positive' 'Negative' 'Neutral']


In [57]:
# repeat the vectorizing steps for test set
df_test['tokens'] = df_test['OriginalTweet'].apply(tokenize)
df_test['tokens_no_url'] = df_test['tokens'].apply(remove_url)
df_test['tokens_no_punc'] = df_test['tokens_no_url'].apply(remove_punc_spec_lower)
df_test['tokens_stemmed'] = df_test['tokens_no_punc'].apply(stemming)
df_test['stems_no_sw'] = df_test['tokens_stemmed'].apply(remove_sw)
df_test['Sentiment_num'] = df_test['Sentiment'].map(encode)
x_test, y_test = df_test['stems_no_sw'].to_numpy(), df_test['Sentiment_num'].to_numpy()

count_vec_test = CountVectorizer(
    analyzer='word',
    tokenizer= override_fcn,
    preprocessor= override_fcn,
    token_pattern= None,
    max_features = 10000)

counts_test = count_vec_test.fit_transform(x_test) # This is the count matrix for the test data
print(counts_test.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [68]:
nb = MultinomialNB(alpha=0.1) # alpha value larger than 1 -> smoother
nb.fit(counts_train, y_train) 

train_preds = nb.predict(counts_train)
test_preds = nb.predict(counts_test)

print('Train accuracy with simple Naive Bayes:', accuracy_score(y_train, train_preds))
print('Test accuracy with simple Naive Bayes:', accuracy_score(y_test, test_preds))

Train accuracy with simple Naive Bayes: 0.770682994387346
Test accuracy with simple Naive Bayes: 0.33438651922064244


In [62]:
df_train_positive = df_train[df_train['Sentiment_num'] == 2]
df_train_neutral = df_train[df_train['Sentiment_num'] == 1]
df_train_negative = df_train[df_train['Sentiment_num'] == 0]

x_train_positive = df_train_positive['stems_no_sw'].to_numpy()
x_train_neutral = df_train_neutral['stems_no_sw'].to_numpy()
x_train_negative = df_train_negative['stems_no_sw'].to_numpy()

lst_positive = [token for row in df_train_positive['stems_no_sw'] for token in row]
lst_neutral = [token for row in df_train_neutral['stems_no_sw'] for token in row]
lst_negative = [token for row in df_train_negative['stems_no_sw'] for token in row]

counts_positive = Counter(lst_positive)
counts_neutral = Counter(lst_neutral)
counts_negative = Counter(lst_negative)

print(counts_positive.most_common(5))
print(counts_neutral.most_common(5))
print(counts_negative.most_common(5))

[('coronaviru', 7467), ('covid19', 6003), ('', 4926), ('store', 3896), ('thi', 3772)]
[('coronaviru', 3792), ('covid19', 2752), ('', 2698), ('store', 1581), ('supermarket', 1436)]
[('coronaviru', 6703), ('covid19', 4862), ('price', 4332), ('', 3667), ('food', 3623)]


**i) Would it be appropriate to fit an ROC curve in this scenario? If yes, explain why. If no, explain why not.**

**j) Redo parts G-H using TF-IDF vectors instead of count vectors. You might find Scikitlearn’s TfidfVectorizer() transformer useful. Report the training and test accuracy. How does this compare to the accuracy using count vectors?**

In [77]:
TFIDF_train = TfidfVectorizer(
    analyzer='word',
    tokenizer= override_fcn,
    preprocessor= override_fcn,
    token_pattern= None,
    max_features = 10000)

TFIDF_test = TfidfVectorizer(
    analyzer='word',
    tokenizer= override_fcn,
    preprocessor= override_fcn,
    token_pattern= None,
    max_features = 10000)

tfs_train = TFIDF_train.fit_transform(x_train)
tfs_test = TFIDF_test.fit_transform(x_test)

matrix_tftrain = tfs_train.toarray()
print(len(matrix_tftrain[0]))

nb1 = MultinomialNB(alpha=0.1)

nb1.fit(tfs_train, y_train)

tf_train_preds = nb1.predict(tfs_train)
tf_test_preds = nb1.predict(tfs_test)

print('Train accuracy with simple Naive Bayes under TF-IDF:', accuracy_score(y_train, tf_train_preds))
print('Test accuracy with simple Naive Bayes under TF-IDF:', accuracy_score(y_test, tf_test_preds))

10000
Train accuracy with simple Naive Bayes under TF-IDF: 0.757465315742158
Test accuracy with simple Naive Bayes under TF-IDF: 0.37519747235387046


In [69]:
lst_positive = [token for row in df_train_positive['stems_no_sw'] for token in row]
lst_neutral = [token for row in df_train_neutral['stems_no_sw'] for token in row]
lst_negative = [token for row in df_train_negative['stems_no_sw'] for token in row]

counts_positive = Counter(lst_positive)
counts_neutral = Counter(lst_neutral)
counts_negative = Counter(lst_negative)

print(counts_positive.most_common(5))
print(counts_neutral.most_common(5))
print(counts_negative.most_common(5))

[('coronaviru', 7467), ('covid19', 6003), ('', 4926), ('store', 3896), ('thi', 3772)]
[('coronaviru', 3792), ('covid19', 2752), ('', 2698), ('store', 1581), ('supermarket', 1436)]
[('coronaviru', 6703), ('covid19', 4862), ('price', 4332), ('', 3667), ('food', 3623)]


**k) Recall lemmatization converts each word to its base form, which is a bit stronger than simply taking the stem. Redo parts E-H using TF-IDF vectors instead of count vectors. This time use lemmatization instead of stemming. Report train and test accuracy. How does the accuracy with lemmatization compare to the accuracy with stemming?**

In [72]:
lemmatizer = WordNetLemmatizer()

def lem(lst):
  return [lemmatizer.lemmatize(t) for t in lst]

TFIDF_train1 = TfidfVectorizer(
    analyzer='word',
    tokenizer= override_fcn,
    preprocessor= override_fcn,
    token_pattern= None,
    max_features = 10000)

TFIDF_test1 = TfidfVectorizer(
    analyzer='word',
    tokenizer= override_fcn,
    preprocessor= override_fcn,
    token_pattern= None,
    max_features = 10000)

# process train data
df_train['lem_tokens'] = df_train['tokens_no_punc'].apply(lem)
df_train['lems_no_sw'] = df_train['lem_tokens'].apply(remove_sw)
x_train_1, y_train_1 = df_train['lems_no_sw'].to_numpy(), df_train['Sentiment_num'].to_numpy()
tfs_train_1 = TFIDF_train1.fit_transform(x_train_1)

In [73]:
# process test data
df_test['lem_tokens'] = df_test['tokens_no_punc'].apply(lem)
df_test['lems_no_sw'] = df_test['lem_tokens'].apply(remove_sw)
x_test_1, y_test_1 = df_test['lems_no_sw'].to_numpy(), df_test['Sentiment_num'].to_numpy()
tfs_test_1 = TFIDF_test1.fit_transform(x_test_1)

In [76]:
nb2 = MultinomialNB(alpha=0.1)

nb2.fit(tfs_train_1, y_train_1)

tf_train_preds_1 = nb2.predict(tfs_train_1)
tf_test_preds_1 = nb2.predict(tfs_test_1)

print('Train accuracy with simple Naive Bayes under TF-IDF and lemmatization:', accuracy_score(y_train_1, tf_train_preds_1))
print('Test accuracy with simple Naive Bayes under TF-IDF and lemmatization:', accuracy_score(y_test_1, tf_test_preds_1))

Train accuracy with simple Naive Bayes under TF-IDF and lemmatization: 0.7678402215904949
Test accuracy with simple Naive Bayes under TF-IDF and lemmatization: 0.3907319641916798


In [75]:
df_train_positive_1 = df_train[df_train['Sentiment_num'] == 2]
df_train_neutral_1 = df_train[df_train['Sentiment_num'] == 1]
df_train_negative_1 = df_train[df_train['Sentiment_num'] == 0]

x_train_positive_1 = df_train_positive_1['lems_no_sw'].to_numpy()
x_train_neutral_1 = df_train_neutral_1['lems_no_sw'].to_numpy()
x_train_negative_1 = df_train_negative_1['lems_no_sw'].to_numpy()

lst_positive = [token for row in df_train_positive_1['lems_no_sw'] for token in row]
lst_neutral = [token for row in df_train_neutral_1['lems_no_sw'] for token in row]
lst_negative = [token for row in df_train_negative_1['lems_no_sw'] for token in row]

counts_positive = Counter(lst_positive)
counts_neutral = Counter(lst_neutral)
counts_negative = Counter(lst_negative)

print(counts_positive.most_common(5))
print(counts_neutral.most_common(5))
print(counts_negative.most_common(5))

[('coronavirus', 7464), ('covid19', 5995), ('', 4926), ('store', 3891), ('price', 3278)]
[('coronavirus', 3792), ('covid19', 2736), ('', 2698), ('store', 1579), ('supermarket', 1436)]
[('coronavirus', 6701), ('covid19', 4858), ('price', 4289), ('', 3667), ('food', 3623)]


**Bonus: Is the Naive Bayes model generative or discriminative? Explain your response.**

# Part 2 Having fun with NLP using the Twitter API